In [5]:
import tkinter as tk
from tkinter import messagebox
from sklearn.linear_model import LinearRegression
import numpy as np

# Dữ liệu mẫu với các điểm thành phần (chuyên cần, bài tập, kiểm tra) và điểm tổng kết
data = {
    'attendance_score': [7, 8, 6, 9, 5, 7.5, 8, 9, 6, 10],
    'homework_score': [6, 7, 5, 8, 6, 7, 8, 7, 5, 9],
    'test_score': [7, 6, 8, 7.5, 5, 6, 7.5, 9, 6, 8],
    'final_score': [6.5, 7.2, 6.8, 8.0, 5.0, 7.0, 8.0, 8.5, 6.0, 9.0]
}

# Chuẩn bị dữ liệu cho mô hình
X = np.array([data['attendance_score'], data['homework_score'], data['test_score']]).T
y = np.array(data['final_score'])

# Huấn luyện mô hình
model = LinearRegression()
model.fit(X, y)

# Hàm phân loại điểm thi
def classify_score(score):
    if score >= 9:
        return 'A+'
    elif 8.5 <= score < 9:
        return 'A'
    elif 8 <= score < 8.5:
        return 'B+'
    elif 7 <= score < 8:
        return 'B'
    elif 6 <= score < 7:
        return 'C+'
    elif 5 <= score < 6:
        return 'C'
    elif 4.5 <= score < 5:
        return 'D+'
    elif 4 <= score < 4.5:
        return 'D'
    else:
        return 'Trung bình'

# Hàm dự đoán điểm tổng kết và phân loại
def predict_and_classify():
    try:
        # Lấy dữ liệu từ các Entry và chuyển đổi sang dạng float
        attendance = float(attendance_entry.get())
        homework = float(homework_entry.get())
        test = float(test_entry.get())
        
        # Dự đoán điểm tổng kết
        input_data = np.array([[attendance, homework, test]])
        final_score_pred = model.predict(input_data)[0]
        
        # Phân loại điểm
        grade = classify_score(final_score_pred)
        
        # Hiển thị kết quả
        result_text = f"Dự đoán điểm tổng kết: {final_score_pred:.2f}\nPhân loại: {grade}"
        messagebox.showinfo("Kết quả", result_text)
    except ValueError:
        messagebox.showerror("Lỗi", "Vui lòng nhập các giá trị hợp lệ.")

# Tạo giao diện người dùng với tkinter
app = tk.Tk()
app.title("Dự đoán và Phân loại Điểm tổng kết")
app.geometry("300x300")

# Nhãn và trường nhập cho từng thành phần điểm
tk.Label(app, text="Điểm chuyên cần (10%):").pack(pady=5)
attendance_entry = tk.Entry(app)
attendance_entry.pack(pady=5)

tk.Label(app, text="Điểm bài tập (10%):").pack(pady=5)
homework_entry = tk.Entry(app)
homework_entry.pack(pady=5)

tk.Label(app, text="Điểm kiểm tra (20%):").pack(pady=5)
test_entry = tk.Entry(app)
test_entry.pack(pady=5)

# Nút dự đoán
predict_button = tk.Button(app, text="Dự đoán Điểm tổng kết và Phân loại", command=predict_and_classify)
predict_button.pack(pady=20)

app.mainloop()


In [12]:
import pandas as pd

# Load the dataset to examine its structure and first few rows
file_path = 'nhadathadong.csv'
data = pd.read_csv(file_path)

# Display the first few rows and column information
data.head(), data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   area              71 non-null     object 
 1   areaM2            61 non-null     float64
 2   balconyDirection  19 non-null     object 
 3   bedroom           41 non-null     object 
 4   bedroomCount      41 non-null     object 
 5   direction         34 non-null     object 
 6   floorCount        41 non-null     object 
 7   frontage          40 non-null     object 
 8   lat               71 non-null     float64
 9   legal             65 non-null     object 
 10  long              71 non-null     float64
 11  price             71 non-null     object 
 12  priceBil          58 non-null     float64
 13  priceExt          62 non-null     object 
 14  priceMil          71 non-null     int64  
 15  pricePerM2        61 non-null     float64
 16  priceVnd          71 non-null     int64  
 17 

(      area  areaM2 balconyDirection bedroom bedroomCount  direction  \
 0    92 m²    92.0              NaN    3 PN      3 phòng        NaN   
 1   100 m²   100.0              NaN     NaN          NaN        NaN   
 2    80 m²    80.0              NaN     NaN          NaN  Tây - Bắc   
 3    60 m²    60.0              NaN     NaN          NaN  Tây - Bắc   
 4  75,3 m²     NaN              NaN     NaN          NaN        NaN   
 
   floorCount frontage        lat             legal        long  \
 0        NaN      NaN  20.979528  Hợp đồng mua bán  105.787567   
 1        NaN      5 m  20.979240               NaN  105.741783   
 2        NaN      4 m  20.979240             Sổ đỏ  105.741783   
 3        NaN      5 m  20.968100    Sổ đỏ/ Sổ hồng  105.754989   
 4        NaN      NaN  20.984566               NaN  105.785019   
 
             price  priceBil          priceExt  priceMil  pricePerM2  \
 0          3,4 tỷ       3.4   ~36,96 triệu/m²      3400   36.956522   
 1           20 tỷ

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Data preprocessing steps
# Selecting relevant columns and dropping rows with missing target price
relevant_data = data[['areaM2', 'bedroomCount', 'floorCount', 'direction', 'priceMil', 'toiletCount']].dropna(subset=['priceMil'])
relevant_data = relevant_data.fillna(0)  # Fill NaN in other columns with 0 for simplicity

# Encoding categorical columns (bedroomCount, direction, toiletCount)
label_encoders = {}
for column in ['bedroomCount', 'direction', 'toiletCount']:
    label_encoders[column] = LabelEncoder()
    relevant_data[column] = label_encoders[column].fit_transform(relevant_data[column].astype(str))

# Defining features and target
X = relevant_data[['areaM2', 'bedroomCount', 'floorCount', 'direction', 'toiletCount']]
y = relevant_data['priceMil']

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predicting and evaluating
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mse, r2


ValueError: could not convert string to float: '4 tầng'